In [46]:


import numpy as np
import platform
import tempfile
import os
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import io, color
from scipy.fftpack import dct



In [47]:
im=io.imread('pelican99.jpg')
im = color.rgb2ycbcr(im)


In [48]:

def votesforeachpixel(im) :
    shape = im.shape
    X = shape[0]
    Y = shape[1]  
    zeros = np.zeros((X,Y))
    votes = np.zeros((X,Y))
    for x in range(X-7) :
        for y in range(Y-7) :
            block = im[x:x+8,y:y+8,0]
            constantx = False
            constanty = False

            #on regarde si le block est constant selon x ou y
            for xx in range(x,x+8) :
                for yy in range(y,y+8) :
                    if block[xx,yy] == block [xx,y] :
                        constanty = True

                    if block[xx,yy] == block [x,yy] :
                        constantx = True

            dctblock = dct(dct(block, axis=0, norm='ortho'), axis=1, norm='ortho')
            nbzeros = 0

            for i in range(8) :
                for j in range(8) :
                    if abs(dctblock[i,j])<0.5 :
                        nbzeros+=1
            for xx in range(8) :
                for yy in range(8) :
                    if nbzeros>zeros[x+xx,y+yy] :
                        zeros[x+xx,y+yy] = nbzeros
                        if not (constantx or constanty) :
                            votes[x+xx,y+yy]= (x%8,y%8)
    return votes

In [49]:
def detectgrid (im) :
    shape = im.shape
    X = shape[0]
    Y = shape[1]  
    votes = votesforeachpixel(im)
    gridvotes = np.zeros(8,8)
    maxvote =0
    mostvotedgrid = 0
    for x in range(X) :
        for y in range(Y) :
            if votes[x,y] != 0 :
                gridvotes[votes[x,y]]+=1
                if gridvotes[votes[x,y]]> maxvote :
                    maxvote = gridvotes[votes[x,y]]
                    mostvotedgrid = votes[x,y]
    # il reste à coder le calcul du nfa
    return mostvotedgrid

In [50]:
print(detectgrid(im))

IndexError: index 8 is out of bounds for axis 1 with size 8